# 12.21 Agricultural pricing
Williams, H. Paul. Model building in mathematical programming. John Wiley & Sons, 2013.

In [ ]:
import numpy as np
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

## 問題設定
国の政府は、乳製品、牛乳、バター、チーズにどの価格を請求するかを決定したいと考えています。これらすべての製品は、国の生乳生産から直接的または間接的に生まれています。この生乳は、脂肪と乾物という2つの成分に分けられます。農場での輸出または消費用の製品の製造に使用される脂肪と乾物の量を差し引いた後、年間合計で60万トンの脂肪と75万トンの乾物が利用可能です。これはすべて国内消費用の牛乳、バター、2種類のチーズの生産に利用できます。

消費者の需要を各製品の価格に関連付ける需要の価格弾力性は、過去の統計に基づいて計算されています。 製品の価格弾力性Eは、

$$
E_{AB} = \frac{\text{Percentage decrease in demand}}{\text{Percentage increase in price}}.
$$

2種類のチーズについては、相対価格に応じて、消費者の需要にある程度の代替があります。 これは、価格に対する需要の交差弾性によって測定されます。 製品Aから製品Bへの交差弾性EABは、

$$
E_{AB} = \frac{\text{Percentage increase in demand for A}}{\text{Percentage increase in price of B}}.
$$

目的関数：
目的は、どの価格とその結果の需要が総収入を最大化するかを決定することです。

制約条件：
しかし、特定の物価指数の上昇を認めることは政治的に受け入れられません。 この物価指数の計算方法の結果として、この制限は、新しい価格が昨年の消費の総コストが増加しないようなものでなければならないことを単に要求しています。 特に重要な追加要件は、この政治的制限の経済的コストを定量化することです。

## 集合の読み込み

In [ ]:
INGREDIENTS = ["Fat", "Dry matter", "Water"]
PRODUCTS = ["Milk", "Butter", "Cheese 1", "Cheese 2"]

## パラメータの読み込み

In [ ]:
availabilities = pd.Series([600000, 750000, 1e+10], index=INGREDIENTS)

In [ ]:
composition = pd.DataFrame([
    [4, 9, 87],
    [80, 2, 18],
    [35, 30, 35],
    [25, 40, 35],
], columns=INGREDIENTS, index=PRODUCTS)
composition

In [ ]:
demand_prev = pd.Series([4820, 320, 210, 70], index=PRODUCTS)
price_prev = pd.Series([297, 720, 1050, 815], index=PRODUCTS)

In [ ]:
EL = pd.Series([0.4, 2.7, 1.1, 0.4], index=PRODUCTS)
CEL = pd.DataFrame([
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0.1],
    [0, 0, 0.4, 0],
], index=PRODUCTS, columns=PRODUCTS)
CEL

### 変数

In [ ]:
price = pd.Series(LpVariable.dicts("price", PRODUCTS, lowBound=0))

In [ ]:
demand = demand_prev.copy()
demand -= demand_prev * EL * (price - price_prev) / price_prev * 100
demand += demand_prev * sum(CEL.loc[:, p] * (price[p] - price_prev[p]) / price_prev[p] * 100
                            for p in PRODUCTS)

### 目的関数

In [ ]:
# 二次計画
total_sales = (price * demand).sum()
total_sales

In [ ]:
model = LpProblem("Agricultural_pricing", LpMaximize)
model.setObjective(total_sales)

### 制約条件

In [ ]:
# 素材の利用可能量
consumption_ingredient = demand.dot(composition)
for i in INGREDIENTS:
    model.addConstraint(consumption_ingredient[i] <= availabilities[i])

In [ ]:
# 昨年の消費に対する総コスト
total_sales_increase = ((price - price_prev) * demand_prev).sum()
model.addConstraint(total_sales_increase <= 0)

In [ ]:
model.solve()

In [ ]:
model